In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

In [3]:
df = pd.read_csv('revenue_data.csv')
df.head()


,transaction_id,date,platform,customer_id,customer_segment,region_city,product_id,product_category,order_status,number_of_orders,...,platform_commission,competitor_price,weather_condition,peak_season_flag,customer_satisfaction,review_count,courier_reliability,market_demand_index,estimated_financial_loss,delivery_success_rate
0,TRX-2023-001,16/08/2023,Shopee,CUST-001,Regular,Bacolod,PROD-Q4,Electronics,Failed,70,...,5.3,142923,normal,0,3.6,75,8.3,9,1775.19,92.10
1,TRX-2023-002,22/02/2023,Shopee,CUST-002,Regular,Manila,PROD-Y5,Health & Beauty,Completed,108,...,6.1,86986,heavy_rain,1,3.1,30,5.6,1,1006.52,85.55
2,TRX-2023-003,01/06/2023,Amazon,CUST-003,Premium,Iloilo,PROD-R10,Fashion,Completed,147,...,9.2,87915,normal,0,4.6,89,9.2,7,1086.12,85.47
3,TRX-2023-004,09/03/2023,Lazada,CUST-004,New,Davao,PROD-U9,Automotive,Completed,122,...,8.7,121431,typhoon,0,4.3,19,7.3,4,1478.54,86.09
4,TRX-2023-005,04/04/2023,Shopee,CUST-005,Regular,Davao,PROD-N10,Health & Beauty,Completed,120,...,6.6,149898,typhoon,0,5.0,45,7.5,10,1962.42,86.85


In [4]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info)

transaction_id               object
date                         object
platform                     object
customer_id                  object
customer_segment             object
region_city                  object
product_id                   object
product_category             object
order_status                 object
number_of_orders              int64
total_order_value             int64
courier_name                 object
delivery_status              object
delivery_time_days          float64
customer_lifetime_value       int64
acquisition_source           object
acquisition_cost              int64
gross_margin_percentage     float64
campaign_id                  object
discount_percentage         float64
platform_commission         float64
competitor_price              int64
weather_condition            object
peak_season_flag              int64
customer_satisfaction       float64
review_count                  int64
courier_reliability         float64
market_demand_index         

In [5]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("categorical_features:", categorical_features)
print("numerical_features:", numerical_features)


categorical_features: Index(['transaction_id', 'date', 'platform', 'customer_id', 'customer_segment',
       'region_city', 'product_id', 'product_category', 'order_status',
       'courier_name', 'delivery_status', 'acquisition_source', 'campaign_id',
       'weather_condition'],
      dtype='object')
numerical_features: Index(['number_of_orders', 'total_order_value', 'delivery_time_days',
       'customer_lifetime_value', 'acquisition_cost',
       'gross_margin_percentage', 'discount_percentage', 'platform_commission',
       'competitor_price', 'peak_season_flag', 'customer_satisfaction',
       'review_count', 'courier_reliability', 'market_demand_index',
       'estimated_financial_loss', 'delivery_success_rate'],
      dtype='object')


In [6]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display =10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ['...']
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}


print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - transaction_id: ['TRX-2023-001', 'TRX-2023-002', 'TRX-2023-003', 'TRX-2023-004', 'TRX-2023-005', 'TRX-2023-006', 'TRX-2023-007', 'TRX-2023-008', 'TRX-2023-009', 'TRX-2023-010', '...']
 - date: ['16/08/2023', '22/02/2023', '01/06/2023', '09/03/2023', '04/04/2023', '06/12/2023', '21/05/2023', '01/12/2023', '08/05/2023', '10/10/2023', '...']
 - platform: ['Shopee', 'Amazon', 'Lazada', 'Zalora']
 - customer_id: ['CUST-001', 'CUST-002', 'CUST-003', 'CUST-004', 'CUST-005', 'CUST-006', 'CUST-007', 'CUST-008', 'CUST-009', 'CUST-010', '...']
 - customer_segment: ['Regular', 'Premium', 'New']
 - region_city: ['Bacolod', 'Manila', 'Iloilo', 'Davao', 'Cagayan de Oro', 'Cebu']
 - product_id: ['PROD-Q4', 'PROD-Y5', 'PROD-R10', 'PROD-U9', 'PROD-N10', 'PROD-Q1', 'PROD-J4', 'PROD-B10', 'PROD-X1', 'PROD-T7', '...']
 - product_category: ['Electronics', 'Health & Beauty', 'Fashion', 'Automotive', 'Toys']
 - order_status: ['Failed', 'Completed', 'Delayed']
 - cour

In [7]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        number_of_orders  total_order_value  delivery_time_days  \
count        100.000000         100.000000          100.000000   
mean         123.060000      119222.690000            3.529000   
std           46.390921       56409.022753            0.883781   
min           50.000000       28899.000000            2.100000   
25%           76.750000       80953.500000            2.700000   
50%          122.000000      105892.000000            3.500000   
75%          165.000000      151201.500000            4.300000   
max          200.000000      288795.000000            5.000000   

       customer_lifetime_value  acquisition_cost  gross_margin_percentage  \
count               100.000000        100.000000                100.00000   
mean             171889.820000        419.440000                 30.47600   
std               77777.099731        116.464181                  5.66699   
min               50497.000000        209.000000 

In [9]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):
    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   - {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [10]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"In the transaction with ID {transaction_id} on {date}, a total of {number_of_orders} orders were placed on the {platform} platform by customer {customer_id}, who belongs to the {customer_segment} segment in {region_city}. The product with ID {product_id}, categorized under {product_category}, was sold with an order status of {order_status}. The total order value reached {total_order_value}, and the delivery was handled by {courier_name}, achieving a delivery status of {delivery_status} with a delivery time of {delivery_time_days} days. The customer lifetime value was estimated at {customer_lifetime_value}, with an acquisition cost of {acquisition_cost} from the {acquisition_source} source. The campaign ID associated with this transaction is {campaign_id}, which offered a discount of {discount_percentage}%. The platform commission was {platform_commission}%, while the competitor price stood at {competitor_price}. The weather condition during this transaction was {weather_condition}, an

In [11]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'
    paragraph = template.format(**row_dict)
    return paragraph

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph


In [12]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [13]:
df['paragraph'][0]

'"In the transaction with ID TRX-2023-001 on 16/08/2023, a total of 70 orders were placed on the Shopee platform by customer CUST-001, who belongs to the Regular segment in Bacolod. The product with ID PROD-Q4, categorized under Electronics, was sold with an order status of Failed. The total order value reached 91140, and the delivery was handled by CourierY, achieving a delivery status of Success with a delivery time of 3.5 days. The customer lifetime value was estimated at 209432, with an acquisition cost of 314 from the Google source. The campaign ID associated with this transaction is CAMP-Q2-001, which offered a discount of 4.2%. The platform commission was 5.3%, while the competitor price stood at 142923. The weather condition during this transaction was normal, and it was 0 that it fell within the peak season. Customer satisfaction was rated at 3.6 based on 75 reviews, and the courier reliability was measured at 8.3. The market demand index was 9, with an estimated financial los

In [14]:
df.head()

,transaction_id,date,platform,customer_id,customer_segment,region_city,product_id,product_category,order_status,number_of_orders,...,competitor_price,weather_condition,peak_season_flag,customer_satisfaction,review_count,courier_reliability,market_demand_index,estimated_financial_loss,delivery_success_rate,paragraph
0,TRX-2023-001,16/08/2023,Shopee,CUST-001,Regular,Bacolod,PROD-Q4,Electronics,Failed,70,...,142923,normal,0,3.6,75,8.3,9,1775.19,92.10,"""In the transaction with ID TRX-2023-001 on 16..."
1,TRX-2023-002,22/02/2023,Shopee,CUST-002,Regular,Manila,PROD-Y5,Health & Beauty,Completed,108,...,86986,heavy_rain,1,3.1,30,5.6,1,1006.52,85.55,"""In the transaction with ID TRX-2023-002 on 22..."
2,TRX-2023-003,01/06/2023,Amazon,CUST-003,Premium,Iloilo,PROD-R10,Fashion,Completed,147,...,87915,normal,0,4.6,89,9.2,7,1086.12,85.47,"""In the transaction with ID TRX-2023-003 on 01..."
3,TRX-2023-004,09/03/2023,Lazada,CUST-004,New,Davao,PROD-U9,Automotive,Completed,122,...,121431,typhoon,0,4.3,19,7.3,4,1478.54,86.09,"""In the transaction with ID TRX-2023-004 on 09..."
4,TRX-2023-005,04/04/2023,Shopee,CUST-005,Regular,Davao,PROD-N10,Health & Beauty,Completed,120,...,149898,typhoon,0,5.0,45,7.5,10,1962.42,86.85,"""In the transaction with ID TRX-2023-005 on 04..."


In [15]:
df.to_csv('revenue_paragraphs.csv', index=False)